<a href="https://colab.research.google.com/github/adwiteeya-r-paul/Dartmouth-Course-Planning-Chatbot/blob/main/MainColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dartmouth Course Planning Chatbot
**Adwiteeya R. Paul (arp)** [adwiteeya.r.paul.27@dartmouth.edu]

**Description**: This project builds a course planning chatbot that takes user input containing on and off terms, major, distributives etc. in natural language and outputs a table of courses for 4 years.


**Single major handling:** I started this project with a single major, Cognitive Science. Cognitive science is an interdisciplinary major and spans across many departments - making it really to plan one's courses keeping course availability, off terms and distributives in mind. It essentially represents a slightly harder form of the usual struggle of Dartmouth students.

We used spaCy and sqlite for data parsing.

**User input parsing:** Building user input parser is ongoing.



In [ ]:
# installing spacy, numpy, pdfminer
!pip install -U spacy numpy pdfminer.six transformers
!python -m spacy download en_core_web_sm
import spacy
from spacy.pipeline import EntityRuler
import os

# mounting google drive for accessing data
mount = '/content/drive'
from google.colab import drive
drive.mount(mount)

# installing pdfminer for pdf handling
!pip install pdfminer.six
from pdfminer.high_level import extract_text # Import the extraction function

#importing sqlite3 for sql database creation
import sqlite3
import os

#importing csv and pandas for csv file creation
import csv
import pandas as pd


# installing transformers for user input processing
!pip install transformers
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


In [ ]:
#function to do data extraction from pdf using NER


def processdata(file):
# Load the English NLP model *without* the 'ner' component
  nlp = spacy.load("en_core_web_sm", disable=["ner"])

# Add the EntityRuler (this will be the only entity recognition component)
  ruler = nlp.add_pipe("entity_ruler")
  patterns = [
   # New pattern for Discipline Code + Number (e.g., "CLST 6")
    {"label": "COURSE_CODE", "pattern": [{"TEXT": "<"}, {"TEXT": {"REGEX": "[A-Z]{3,4}"}}, {"IS_SPACE": True, "OP": "?"}, {"TEXT": {"REGEX": "\\d+"}}, {"TEXT": ">"}]}, # New pattern for e.g., <COSC 1>


     {"label": "PREREQUISITE", "pattern": [
        {"TEXT": {"REGEX": "^[A-Z]{2,4}$"}}, # Match 2-4 uppercase letters (Discipline Code)
        {"IS_SPACE": True, "OP": "?"},      # Optionally match a space between code and number
        {"TEXT": {"REGEX": "^\\d{1,3}|\d{1,2}\.\d{1,2}$"}}    # Match 1-3 digits (Course Number)
    ]},

    # Keep the original single-token pattern just in case (less likely to match standard text)
    {"label": "TERM", "pattern": [{"LOWER": {"IN": ["fall", "winter", "spring", "summer"]}}]},
    {"label": "DISTRIBUTIVE", "pattern": [{"TEXT": {"REGEX": "^(ART|LIT|TMV|INT|SOC|QDS|SCI|SLA|TAS|TLA)$"}}]},
    {"label": "WORLDCULTURE", "pattern": [{"TEXT": {"REGEX": "^(W|NW|INT)$"}}]},

]

  ruler.add_patterns(patterns)

  pdf_file_path = file
# Define the output directory for spaCy results
  spacy_output_dir = "spacy_entities_output"
  worksheet_text = "" # Initialize to empty

# Extract text from the PDF file
  try:
    worksheet_text = extract_text(pdf_file_path)
    print("Successfully extracted text from the PDF.")
  except FileNotFoundError:
    print(f"Error: PDF file not found at {pdf_file_path}")
  except Exception as e:
    print(f"An error occurred while extracting text from the PDF: {e}")

# Process the extracted text with spaCy if successfully extracted
  if worksheet_text:
    print("\nExtracted Text (first 500 chars):")
    print(worksheet_text[:500])
    print("-" * 20)

    doc = nlp(worksheet_text)

    # Create the output directory if it doesn't exist
    os.makedirs(spacy_output_dir, exist_ok=True)

    # Define the output file path (using the PDF name as base)
    pdf_name = os.path.basename(pdf_file_path)
    output_file_path = os.path.join(spacy_output_dir, f"{os.path.splitext(pdf_name)[0]}_entities.txt")

    # Prepare the entity data for writing
    entity_lines = []
    entity_lines.append("Detected Entities (from EntityRuler only):")
    if doc.ents:
        for ent in doc.ents:
          entity_lines.append(f"{ent.label_}: {ent.text}")
    else:
        entity_lines.append("No entities detected.")

    # Write the entity data to the output file
    try:
        with open(output_file_path, "w", encoding="utf-8") as f:
            f.write("\n".join(entity_lines))
        print(f"\n Detected entities saved to '{output_file_path}'")
    except Exception as e:
        print(f" Error saving entities to file: {e}")

  else:
    print("Could not extract text from the PDF.")

  return output_file_path

In [ ]:
# Database to classify data

def createdatabase(database, file):

  db_path = database
  file_path = file

# Kill any old version
  try:
      os.remove(db_path)
  except FileNotFoundError:
      pass

  conn = sqlite3.connect(db_path, timeout=10, isolation_level=None)  # Timeout added
  cursor = conn.cursor()
  table_name = "my_table"

  cursor.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        Course TEXT PRIMARY KEY,
        Prerequisite TEXT,
        Term TEXT,
        Distributive TEXT
    )
""")

# Start parsing
  course_code = None
  prerequisites = []
  terms = []
  distributives = []

  with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        if line.startswith("COURSE_CODE:"):
            # Save previous course
            if course_code:
                cursor.execute("BEGIN")
                cursor.execute(f"""
                    INSERT OR IGNORE INTO {table_name} (Course, Prerequisite, Term, Distributive)
                    VALUES (?, ?, ?, ?)
                """, (
                    course_code,
                    ', '.join(prerequisites),
                    ', '.join(terms),
                    ', '.join(distributives)
                ))
                cursor.execute("COMMIT")

            # Start new course
            course_code = line.split(":", 1)[1].strip()
            prerequisites, terms, distributives = [], [], []

        elif line.startswith("PREREQUISITE:"):
            prerequisites.append(line.split(":", 1)[1].strip())
        elif line.startswith("TERM:"):
            terms.append(line.split(":", 1)[1].strip())
        elif line.startswith("DISTRIBUTIVE:"):
            distributives.append(line.split(":", 1)[1].strip())

# Insert last course
  if course_code:
    cursor.execute("BEGIN")
    cursor.execute(f"""
        INSERT OR IGNORE INTO {table_name} (Course, Prerequisite, Term, Distributive)
        VALUES (?, ?, ?, ?)
    """, (
        course_code,
        ', '.join(prerequisites),
        ', '.join(terms),
        ', '.join(distributives)
    ))
    cursor.execute("COMMIT")

# Show results, testing
  cursor.execute(f"SELECT * FROM {table_name}")
  rows = cursor.fetchall()
  for row in rows:
    print(row)

  conn.close()
  return db_path


In [ ]:
#converting to csv
def convert_to_csv(database, output):


# Define the database and table names
  db_path = database
  table_name = "my_table"
  csv_output_path = output # Name of the output CSV file

# Connect to the database
  try:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Select all data from the table
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()

    # Get the column names (header)
    header = [description[0] for description in cursor.description]

    # Write the data to a CSV file
    with open(csv_output_path, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)

        # Write the header row
        csv_writer.writerow(header)

        # Write the data rows
        csv_writer.writerows(rows)

    print(f"Successfully exported data from '{table_name}' to '{csv_output_path}'")

  except FileNotFoundError:
    print(f"Error: Database file not found at {db_path}")
  except sqlite3.Error as e:
    print(f"Database error: {e}")
  except Exception as e:
    print(f"An unexpected error occurred: {e}")
  finally:
    # Close the database connection
    if 'conn' in locals() and conn:
        conn.close()

  return csv_output_path

In [ ]:
# function to access certain category
def accesscategory(csv_file, category):
  csv_file_path = csv_file

  try:
    df = pd.read_csv(csv_file_path)

    # --- Select a specific course ---
    # Define the course you want to find
    target_course = "<COSC 30>" # Replace with the actual course code you're looking for

    # Filter the DataFrame to find the row(s) where the 'Course' column matches the target course
    specific_course_row = df[df['Course'] == target_course]

    # Check if the course was found
    if not specific_course_row.empty:
        print(f"Details for course '{target_course}':")
        print(specific_course_row)

        # You can also access specific information from the found row(s)
        # For example, the prerequisite(s):
        prerequisites = specific_course_row['Prerequisite'].values
        print(f"\nPrerequisite(s) for '{target_course}': {prerequisites}")

    else:
        print(f"Course '{target_course}' not found in the DataFrame.")

  except FileNotFoundError:
    print(f"Error: CSV file not found at {csv_file_path}")
  except Exception as e:
    print(f"An error occurred while reading the CSV: {e}")


In [ ]:
# main function: extract text from the pdfs containing department ORC information, prerequisites, focus areas courses
# please upload the file change the path of the data #

##
pre = processdata("/prerequisites.pdf") # change your path here

##
cs = processdata("/cs.pdf")

##
cogs = processdata("/cogs.pdf")

##
psyc = processdata("/psyc.pdf")

##
ling = processdata("/ling.pdf")

##
phil = processdata("/phil.pdf")
deci = processdata("/decision_making.pdf")
lang = processdata("/language_and_thought.pdf")
dev = processdata("/learning_and_development.pdf")
cons = processdata("/consciousness.pdf")
soc = processdata("/social_interaction.pdf")
design = processdata("/design.pdf")
intel = processdata("/intelligence.pdf")
moral= processdata("/moral_reasoning.pdf")

predb = createdatabase("pre_database.db", pre)
inteldb = createdatabase("intel_database.db", intel)

precsv =  convert_to_csv(predb, "pre.csv")
intelcsv = convert_to_csv(inteldb, "intel.csv")




In [ ]:
df_pre = pd.read_csv(precsv)
df_intel = pd.read_csv(intelcsv)
completed = []
thesis_pre = df_pre[
    df_pre['Course'].str.contains('<COGS 86>', na=False) |
    df_pre['Course'].str.contains('<COGS 87>', na=False)
]
research_pre = df_pre[
    df_pre['Course'].str.contains('<COGS 85>', na=False)
]
seminar_pre = df_pre[
    df_pre['Course'].str.contains('<COGS 81>', na=False)
]

def thesis_filter(df):
  plan = df_pre[
    ~df_pre['Course'].str.contains('<COGS 86>', na=False) &
    ~df_pre['Course'].str.contains('<COGS 87>', na=False) &
    ~df_pre['Course'].str.contains('<COGS 85>', na=False) &
    ~df_pre['Course'].str.contains('<COGS 81>', na=False) &
    ~df_pre['Course'].str.contains('<COGS 80>', na=False)
  ]
  return plan


def season_filter(df, df2, season):
  term_condition = (df['Term'].str.contains(season, na=False) | df['Term'].isna()) & (df['Prerequisite'].isna() | df['Prerequisite'].isin(df))
  merged_indicator = df_pre.merge(df2, indicator=True, how='left')
  rows_unique_to_df_condition = merged_indicator['_merge'] == 'left_only'
  combined_condition = term_condition & rows_unique_to_df_condition
  plan = df[combined_condition]
  return plan



df = thesis_filter(df_pre)

fall =  df[df['Term'].str.contains('Fall', na=False) & df['Prerequisite'].isna()]

fall = fall.sample(n=2)
new_row_df = pd.DataFrame([['<WRIT 5>', 'NaN', 'Fall', 'WREQ']], columns=['Course', 'Prerequisite', 'Term', 'Distributive'], index=[fall.shape[0]]) # Replace with data, column names, and desired index
fall = pd.concat([fall, new_row_df], axis = 0)
fall['Term'] = '25F'


winter = season_filter(df, fall, "Winter")
winter = winter.sample(n=2)
new_row_df = pd.DataFrame([['<WRIT 7>', 'WRIT 5', 'Winter', 'WREQ']], columns=['Course', 'Prerequisite', 'Term', 'Distributive'], index=[winter.shape[0]]) # Replace with data, column names, and desired index
winter = pd.concat([winter, new_row_df], axis = 0)
winter['Term'] = '25W'
pd.concat([fall, winter], axis=0)

spring = season_filter(df, winter, "Spring")

spring = spring.sample(n=3)
spring['Term'] = '25S'
pd.concat([fall, winter, spring], axis=0)




